In [1]:
# ==============================================================================
# BƯỚC 0: DỌN DẸP SẠCH SẼ & CÀI ĐẶT
# ==============================================================================
# Xóa các file/folder cũ để tránh nhầm lẫn
!rm -rf tfjs_model tfjs_model_ultra.zip scaler_ecg.json *.h5

# Cài đặt thư viện
!pip install wfdb tensorflowjs scikit-learn matplotlib numpy pandas

import wfdb
import numpy as np
import os
import json
import shutil
from scipy.signal import resample
from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import classification_report
import tensorflow as tf
from tensorflow.keras.layers import (Conv1D, MaxPooling1D, Dense, Flatten, Dropout,
                                     BatchNormalization, Input, Add, Activation,
                                     GlobalAveragePooling1D, SpatialDropout1D)
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import ModelCheckpoint, LearningRateScheduler
from tensorflow.keras.utils import to_categorical

# ==============================================================================
# BƯỚC 1: CẤU HÌNH HỆ THỐNG
# ==============================================================================
TARGET_FS = 125        # Tần số lấy mẫu của AD8232
WINDOW_SIZE = 100      # 100 mẫu (0.8 giây)
SHIFT_RANGE = 4        # Dịch chuyển cửa sổ +/- 4 mẫu để tạo thêm dữ liệu

# ==============================================================================
# BƯỚC 2: TẢI & XỬ LÝ DỮ LIỆU (SHIFT AUGMENTATION)
# ==============================================================================
print("⬇️ Đang tải dữ liệu MIT-BIH...")
wfdb.dl_database('mitdb', dl_dir='mitdb_data')

classes = ['N', 'S', 'V', 'F']
aami_dict = {
    'N': 'N', 'L': 'N', 'R': 'N', 'e': 'N', 'j': 'N',
    'A': 'S', 'a': 'S', 'J': 'S', 'S': 'S',
    'V': 'V', 'E': 'V', 'F': 'F', '/': 'Q', 'f': 'Q', 'Q': 'Q'
}
class_map = {cls: i for i, cls in enumerate(classes)}

data_by_class = {0: [], 1: [], 2: [], 3: []}
records = [f.split('.')[0] for f in os.listdir('mitdb_data') if f.endswith('.dat')]
records = sorted(list(set(records)))

print("⚙️ Đang xử lý tín hiệu & Augmentation...")
for record_name in records:
    # Bỏ qua các bản ghi dùng máy tạo nhịp tim (Paced beats)
    if record_name in ['102', '104', '107', '217']: continue

    path = os.path.join('mitdb_data', record_name)
    try:
        record = wfdb.rdrecord(path)
        annotation = wfdb.rdann(path, 'atr')
        signal = record.p_signal[:, 0] # Lấy Lead II

        # Resample về 125Hz
        num_samples = int(len(signal) * TARGET_FS / record.fs)
        signal = resample(signal, num_samples)
        r_peaks = (annotation.sample * TARGET_FS / record.fs).astype(int)

        for i, r_peak in enumerate(r_peaks):
            label = annotation.symbol[i]
            if label in aami_dict:
                mapped = aami_dict[label]
                if mapped in classes:
                    cls_idx = class_map[mapped]

                    # --- CHIẾN THUẬT AUGMENTATION ---
                    # Class N (0): Chỉ lấy đúng tâm (không nhân bản để tránh quá nhiều)
                    # Class S, V, F: Lấy tâm + Lệch trái + Lệch phải => Nhân 3 dữ liệu
                    offsets = [0]
                    if cls_idx != 0:
                        offsets = [0, -SHIFT_RANGE, SHIFT_RANGE]

                    for offset in offsets:
                        center = r_peak + offset
                        start, end = center - WINDOW_SIZE // 2, center + WINDOW_SIZE // 2
                        if start >= 0 and end <= len(signal):
                            seg = signal[start:end]
                            data_by_class[cls_idx].append(seg)
    except: pass

# ==============================================================================
# BƯỚC 3: CÂN BẰNG & CHUẨN HÓA
# ==============================================================================
X_final, y_final = [], []
MAX_N = 30000 # Tăng giới hạn mẫu N lên để model học kỹ hơn

print("\n📊 Thống kê dữ liệu sau Augmentation:")
for cls_idx, segments in data_by_class.items():
    segments = np.array(segments)
    count = len(segments)
    print(f" - Class {classes[cls_idx]}: {count} mẫu")

    # Giảm mẫu Class N nếu quá nhiều
    if cls_idx == 0 and count > MAX_N:
        indices = np.random.choice(count, MAX_N, replace=False)
        segments = segments[indices]

    X_final.extend(segments)
    y_final.extend([cls_idx] * len(segments))

X_raw = np.array(X_final)
y = np.array(y_final)

# Tính Mean/Std toàn cục
global_mean = np.mean(X_raw)
global_std = np.std(X_raw)
print(f"\n📏 Global Scaler: Mean={global_mean:.4f}, Std={global_std:.4f}")

# Lưu file scaler_ecg.json MỚI
with open('scaler_ecg.json', 'w') as f:
    json.dump({"mean": float(global_mean), "std": float(global_std)}, f)

# Chuẩn hóa dữ liệu
X = (X_raw - global_mean) / (global_std + 1e-6)
X = X.reshape(-1, WINDOW_SIZE, 1)

# Chia tập Train/Test (Test để riêng 15% để chấm điểm cuối cùng)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15, stratify=y, random_state=42)

# One-hot encoding (Bắt buộc cho Label Smoothing)
y_train_cat = to_categorical(y_train)
y_test_cat = to_categorical(y_test)

# Tính trọng số class
class_weights = dict(enumerate(compute_class_weight('balanced', classes=np.unique(y_train), y=y_train)))

# ==============================================================================
# BƯỚC 4: XÂY DỰNG MODEL (DEEP RESNET VỚI SWISH)
# ==============================================================================
def residual_block(x, filters, kernel_size=5, stride=1):
    shortcut = x
    # Layer 1
    x = Conv1D(filters, kernel_size, strides=stride, padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('swish')(x)
    x = SpatialDropout1D(0.1)(x) # SpatialDropout tốt hơn Dropout thường cho 1D

    # Layer 2
    x = Conv1D(filters, kernel_size, strides=1, padding='same')(x)
    x = BatchNormalization()(x)

    # Khớp shortcut nếu kích thước thay đổi
    if stride > 1 or shortcut.shape[-1] != filters:
        shortcut = Conv1D(filters, 1, strides=stride, padding='same')(shortcut)
        shortcut = BatchNormalization()(shortcut)

    x = Add()([x, shortcut])
    x = Activation('swish')(x)
    return x

inp = Input(shape=(WINDOW_SIZE, 1))
x = Conv1D(64, 7, padding='same')(inp)
x = BatchNormalization()(x)
x = Activation('swish')(x)

# Kiến trúc sâu dần
x = residual_block(x, 64, kernel_size=5)
x = MaxPooling1D(2)(x)

x = residual_block(x, 128, kernel_size=5)
x = MaxPooling1D(2)(x)

x = residual_block(x, 256, kernel_size=3)
x = residual_block(x, 256, kernel_size=3)
x = MaxPooling1D(2)(x)

x = residual_block(x, 512, kernel_size=3)
x = GlobalAveragePooling1D()(x)

x = Dense(256, activation='swish')(x)
x = Dropout(0.5)(x)
out = Dense(4, activation='softmax')(x)

model = Model(inp, out)

# ==============================================================================
# BƯỚC 5: TRAINING (COSINE DECAY + LABEL SMOOTHING)
# ==============================================================================
# Hàm giảm learning rate hình sin
def cosine_decay(epoch):
    lr_start = 0.001
    lr_min = 0.00001
    epochs = 60
    return lr_min + 0.5 * (lr_start - lr_min) * (1 + np.cos(epoch / epochs * np.pi))

lr_scheduler = LearningRateScheduler(cosine_decay)
checkpoint = ModelCheckpoint('best_ultra_model.h5', monitor='val_accuracy', save_best_only=True, verbose=1)

# Label Smoothing = 0.1 giúp model khái quát hóa tốt hơn
model.compile(optimizer='adam',
              loss=tf.keras.losses.CategoricalCrossentropy(label_smoothing=0.1),
              metrics=['accuracy'])

print("\n🚀 Bắt đầu Training...")
history = model.fit(
    X_train, y_train_cat,
    epochs=60,
    batch_size=128,
    validation_split=0.15, # Trích thêm 15% từ train để làm validation
    class_weight=class_weights,
    callbacks=[checkpoint, lr_scheduler],
    verbose=1
)

# ==============================================================================
# BƯỚC 6: ĐÁNH GIÁ & XUẤT FILE (SỬA LỖI KERAS 3)
# ==============================================================================
print("\n🏆 Đánh giá trên tập Test (Dữ liệu chưa từng học):")
model.load_weights('best_ultra_model.h5')

# Dự đoán
y_pred_probs = model.predict(X_test)
y_pred = np.argmax(y_pred_probs, axis=1)
y_true = np.argmax(y_test_cat, axis=1)

# Báo cáo chi tiết (4 chữ số thập phân)
print(classification_report(y_true, y_pred, target_names=classes, digits=4))

# Convert sang TFJS (Cách mới cho Keras 3)
print("\n📦 Đang đóng gói model...")

# 1. Export ra SavedModel (Thay vì .h5)
# Điều này tạo ra một thư mục 'saved_model_ecg' chứa cấu trúc graph chuẩn của TF
model.export('saved_model_ecg')

# 2. Sử dụng tensorflowjs_converter với input_format=tf_saved_model
# output_format=tfjs_graph_model là định dạng tối ưu nhất cho web
!tensorflowjs_converter \
    --input_format=tf_saved_model \
    --output_format=tfjs_graph_model \
    saved_model_ecg \
    ./tfjs_model

# Nén file để tải về
!zip -r tfjs_model_ultra.zip tfjs_model scaler_ecg.json

print("\n✅ HOÀN TẤT! Hãy tải file 'tfjs_model_ultra.zip' về.")

⬇️ Đang tải dữ liệu MIT-BIH...
Generating record list for: 100
Generating record list for: 101
Generating record list for: 102
Generating record list for: 103
Generating record list for: 104
Generating record list for: 105
Generating record list for: 106
Generating record list for: 107
Generating record list for: 108
Generating record list for: 109
Generating record list for: 111
Generating record list for: 112
Generating record list for: 113
Generating record list for: 114
Generating record list for: 115
Generating record list for: 116
Generating record list for: 117
Generating record list for: 118
Generating record list for: 119
Generating record list for: 121
Generating record list for: 122
Generating record list for: 123
Generating record list for: 124
Generating record list for: 200
Generating record list for: 201
Generating record list for: 202
Generating record list for: 203
Generating record list for: 205
Generating record list for: 207
Generating record list for: 208
Generatin

349/349 ━━━━━━━━━━━━━━━━━━━━ 53s 75ms/step - accuracy: 0.7767 - loss: 0.7929 - val_accuracy: 0.7905 - val_loss: 0.7857 - learning_rate: 0.0010
Epoch 2/60
348/349 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - accuracy: 0.9364 - loss: 0.5176
Epoch 2: val_accuracy improved from 0.79053 to 0.95049, saving model to best_ultra_model.h5


349/349 ━━━━━━━━━━━━━━━━━━━━ 8s 21ms/step - accuracy: 0.9365 - loss: 0.5175 - val_accuracy: 0.9505 - val_loss: 0.4695 - learning_rate: 9.9932e-04
Epoch 3/60
348/349 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - accuracy: 0.9578 - loss: 0.4592
Epoch 3: val_accuracy improved from 0.95049 to 0.95773, saving model to best_ultra_model.h5


349/349 ━━━━━━━━━━━━━━━━━━━━ 8s 22ms/step - accuracy: 0.9578 - loss: 0.4591 - val_accuracy: 0.9577 - val_loss: 0.4561 - learning_rate: 9.9729e-04
Epoch 4/60
348/349 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - accuracy: 0.9633 - loss: 0.4488
Epoch 4: val_accuracy improved from 0.95773 to 0.97144, saving model to best_ultra_model.h5


349/349 ━━━━━━━━━━━━━━━━━━━━ 8s 23ms/step - accuracy: 0.9633 - loss: 0.4488 - val_accuracy: 0.9714 - val_loss: 0.4254 - learning_rate: 9.9391e-04
Epoch 5/60
347/349 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - accuracy: 0.9707 - loss: 0.4212
Epoch 5: val_accuracy did not improve from 0.97144
349/349 ━━━━━━━━━━━━━━━━━━━━ 7s 21ms/step - accuracy: 0.9707 - loss: 0.4211 - val_accuracy: 0.9292 - val_loss: 0.5156 - learning_rate: 9.8918e-04
Epoch 6/60
348/349 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - accuracy: 0.9743 - loss: 0.4125
Epoch 6: val_accuracy improved from 0.97144 to 0.97588, saving model to best_ultra_model.h5


...

349/349 ━━━━━━━━━━━━━━━━━━━━ 8s 23ms/step - accuracy: 0.9997 - loss: 0.3548 - val_accuracy: 0.9923 - val_loss: 0.3656 - learning_rate: 2.5750e-04
Epoch 42/60
349/349 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - accuracy: 0.9996 - loss: 0.3508
Epoch 42: val_accuracy improved from 0.99226 to 0.99238, saving model to best_ultra_model.h5


349/349 ━━━━━━━━━━━━━━━━━━━━ 8s 22ms/step - accuracy: 0.9996 - loss: 0.3509 - val_accuracy: 0.9924 - val_loss: 0.3654 - learning_rate: 2.3540e-04
Epoch 43/60
348/349 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step - accuracy: 0.9997 - loss: 0.3525
Epoch 43: val_accuracy did not improve from 0.99238
349/349 ━━━━━━━━━━━━━━━━━━━━ 8s 22ms/step - accuracy: 0.9997 - loss: 0.3525 - val_accuracy: 0.9920 - val_loss: 0.3660 - learning_rate: 2.1405e-04
Epoch 44/60
349/349 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - accuracy: 0.9998 - loss: 0.3515
Epoch 44: val_accuracy did not improve from 0.99238
349/349 ━━━━━━━━━━━━━━━━━━━━ 8s 22ms/step - accuracy: 0.9998 - loss: 0.3515 - val_accuracy: 0.9924 - val_loss: 0.3650 - learning_rate: 1.9349e-04
Epoch 45/60
349/349 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - accuracy: 1.0000 - loss: 0.3536
Epoch 45: val_accuracy improved from 0.99238 to 0.99302, saving model to best_ultra_model.h5


349/349 ━━━━━━━━━━━━━━━━━━━━ 8s 22ms/step - accuracy: 1.0000 - loss: 0.3536 - val_accuracy: 0.9930 - val_loss: 0.3640 - learning_rate: 1.7378e-04
Epoch 46/60
348/349 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - accuracy: 1.0000 - loss: 0.3540
Epoch 46: val_accuracy did not improve from 0.99302
349/349 ━━━━━━━━━━━━━━━━━━━━ 8s 22ms/step - accuracy: 1.0000 - loss: 0.3540 - val_accuracy: 0.9930 - val_loss: 0.3647 - learning_rate: 1.5498e-04
Epoch 47/60
349/349 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - accuracy: 0.9999 - loss: 0.3542
Epoch 47: val_accuracy did not improve from 0.99302
349/349 ━━━━━━━━━━━━━━━━━━━━ 7s 21ms/step - accuracy: 0.9999 - loss: 0.3542 - val_accuracy: 0.9916 - val_loss: 0.3683 - learning_rate: 1.3714e-04
Epoch 48/60
347/349 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step - accuracy: 0.9999 - loss: 0.3542
Epoch 48: val_accuracy did not improve from 0.99302
349/349 ━━━━━━━━━━━━━━━━━━━━ 8s 22ms/step - accuracy: 0.9999 - loss: 0.3542 - val_accuracy: 0.9929 - val_loss: 0.3645 - learning_rate: 1.2031e

349/349 ━━━━━━━━━━━━━━━━━━━━ 8s 22ms/step - accuracy: 1.0000 - loss: 0.3518 - val_accuracy: 0.9933 - val_loss: 0.3645 - learning_rate: 8.9858e-05
Epoch 51/60
348/349 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step - accuracy: 1.0000 - loss: 0.3534
Epoch 51: val_accuracy did not improve from 0.99327
349/349 ━━━━━━━━━━━━━━━━━━━━ 8s 22ms/step - accuracy: 1.0000 - loss: 0.3534 - val_accuracy: 0.9929 - val_loss: 0.3637 - learning_rate: 7.6317e-05
Epoch 52/60
347/349 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - accuracy: 1.0000 - loss: 0.3530
Epoch 52: val_accuracy did not improve from 0.99327
349/349 ━━━━━━━━━━━━━━━━━━━━ 8s 22ms/step - accuracy: 1.0000 - loss: 0.3530 - val_accuracy: 0.9931 - val_loss: 0.3643 - learning_rate: 6.3952e-05
Epoch 53/60
347/349 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step - accuracy: 1.0000 - loss: 0.3541
Epoch 53: val_accuracy improved from 0.99327 to 0.99353, saving model to best_ultra_model.h5


349/349 ━━━━━━━━━━━━━━━━━━━━ 8s 22ms/step - accuracy: 1.0000 - loss: 0.3541 - val_accuracy: 0.9935 - val_loss: 0.3640 - learning_rate: 5.2795e-05
Epoch 54/60
348/349 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - accuracy: 1.0000 - loss: 0.3502
Epoch 54: val_accuracy did not improve from 0.99353
349/349 ━━━━━━━━━━━━━━━━━━━━ 8s 22ms/step - accuracy: 1.0000 - loss: 0.3502 - val_accuracy: 0.9928 - val_loss: 0.3642 - learning_rate: 4.2878e-05
Epoch 55/60
348/349 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - accuracy: 1.0000 - loss: 0.3516
Epoch 55: val_accuracy did not improve from 0.99353
349/349 ━━━━━━━━━━━━━━━━━━━━ 7s 21ms/step - accuracy: 1.0000 - loss: 0.3516 - val_accuracy: 0.9928 - val_loss: 0.3644 - learning_rate: 3.4227e-05
Epoch 56/60
348/349 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - accuracy: 1.0000 - loss: 0.3540
Epoch 56: val_accuracy did not improve from 0.99353
349/349 ━━━━━━━━━━━━━━━━━━━━ 8s 22ms/step - accuracy: 1.0000 - loss: 0.3540 - val_accuracy: 0.9934 - val_loss: 0.3641 - learning_rate: 2.6867e